In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class ResBlk(nn.Module):
    
    def __init__(self, ch_in, ch_out, stride=1):
        super(ResBlk, self).__init__()
        
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)
        
        self.extra = nn.Sequential()
        
        if ch_in != ch_out :
            print("in",ch_in)
            print("out", ch_out)
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=stride),
                nn.BatchNorm2d(ch_out)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        print("x shape", x.shape)
        print("out.shape", out.shape)
        a = self.extra(x)
        print("out.shape2", a.shape)
        out = a + out
        print("out.shape3", out.shape)
        out = F.relu(out)
        
        return out
        
class ResNet18(nn.Module):
    
    def __init__(self):
        super(ResNet18, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=0),
            #nn.Conv2d(3, 64, kernel_size=3, stride=3, padding=0),
            nn.BatchNorm2d(64)
        )
        self.blk1 = ResBlk(64, 128, stride=2)
        self.blk2 = ResBlk(128, 256, stride=2)
        self.blk3 = ResBlk(256, 512, stride=2)
        self.blk4 = ResBlk(512, 512, stride=2)
        self.out_layer = nn.Linear(512*1*1, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.blk1(x)
        x = self.blk2(x)
        x = self.blk3(x)
        x = self.blk4(x)
        
        x = F.adaptive_avg_pool2d(x, [1, 1])
        x = x.view(x.size(0), -1)
        x = self.out_layer(x)
        
        return x
        
        
        
def main():
    #blk = ResBlk(64, 128, stride=4)
    #tmp = torch.randn(2, 64, 56, 56)
    #out = blk(tmp)
    #print('block shape:', out.shape)
    
    x = torch.randn(2, 3, 224, 224)
    model = ResNet18()
    out = model(x)
    #print('out shape:', out.shape)
    
if __name__ == "__main__":
    main()
    

in 64
out 128
in 128
out 256
in 256
out 512
x shape torch.Size([2, 64, 109, 109])
out.shape torch.Size([2, 128, 55, 55])
out.shape2 torch.Size([2, 128, 55, 55])
out.shape3 torch.Size([2, 128, 55, 55])
x shape torch.Size([2, 128, 55, 55])
out.shape torch.Size([2, 256, 28, 28])
out.shape2 torch.Size([2, 256, 28, 28])
out.shape3 torch.Size([2, 256, 28, 28])
x shape torch.Size([2, 256, 28, 28])
out.shape torch.Size([2, 512, 14, 14])
out.shape2 torch.Size([2, 512, 14, 14])
out.shape3 torch.Size([2, 512, 14, 14])
x shape torch.Size([2, 512, 14, 14])
out.shape torch.Size([2, 512, 7, 7])
out.shape2 torch.Size([2, 512, 14, 14])


RuntimeError: The size of tensor a (14) must match the size of tensor b (7) at non-singleton dimension 3